In [2]:
def read_BLOSUM_table(path):
    """
    Reads BLOSUM table
    the key = row_alpha + col_alpha
    """
    fr = open(path,"r")
    res_mat = {}
    alpha = fr.readline().strip().split()
    for line in fr.readlines():
        row_alpha = line[0]
        for col_alpha, val in zip(alpha,line.split()[1:]):
            key = row_alpha + col_alpha
            res_mat[key] = int(val)
    return res_mat

In [3]:
path = "/Users/marcisin/Desktop/BLOSUM62.txt"
table = read_BLOSUM_table(path)

In [166]:
def ConstructGlobalAllignment(backtrack,seq1,seq2):
    """
    Constructs globall allignment from the allignment graph
    """
    ALL_1=""
    ALL_2=""
    i = len(seq1)
    j = len(seq2)
    while i > 0 and j > 0:
        if backtrack[i][j] == "DG":
            ALL_1+=seq1[i-1]
            ALL_2+=seq2[j-1]
            i = i-1
            j = j-1
        # Down
        elif backtrack[i][j] == "D":
            ALL_1+=seq1[i-1]
            ALL_2+="-"
            i = i-1
        else:
            ALL_1+="-"
            ALL_2+=seq2[j-1]
            j = j-1
            
   # because we want full allignment, we need to get to point 0,0
    while i != 0 or j != 0:
        if backtrack[i]!=0:
            ALL_1+=seq1[i-1]
            ALL_2+="-"
            i = i-1
        else:
            ALL_1+="-"
            ALL_2+=seq2[j-1]
            j = j-1
    
    print(ALL_1[::-1])
    print(ALL_2[::-1])

In [186]:
def GlobalAllignmentGraph(seq1, seq2, table, indel_penalty):
    """
    Creates 
    seq1 -> represents rowumn
    seq2 -> represents col
    Returns (path_graph, max_score)
    """
    # need to initialize with indel penalty
    match_graph = [ [0 for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    for row in range(0, len(seq1)+1):
        match_graph[row][0] = -indel_penalty*row  
    for col in range(0, len(seq2)+1):
        match_graph[0][col] = -indel_penalty*col
    
    
    bt_graph = [ ["-" for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    for row in range(1, len(seq1)+1):
        for col in range(1, len(seq2)+1):
            key = seq1[row-1] + seq2[col-1]
            match_score = table[key]
            
            match_graph[row][col] = max(match_graph[row-1][col] - indel_penalty, match_graph[row][col-1] - indel_penalty, match_graph[row-1][col-1]+match_score)
            if match_graph[row][col] == match_graph[row-1][col] - indel_penalty:
                bt_graph[row][col] = "D"
            elif match_graph[row][col] == match_graph[row][col-1] - indel_penalty:
                bt_graph[row][col] = "R"
            elif match_graph[row][col] == match_graph[row-1][col-1]+match_score:
                bt_graph[row][col] = "DG"
    
#     print(match_graph)
    
    return bt_graph, match_graph[-1][-1]

In [185]:
s1="PLEASANTLY"
s2="MEANLY"

g,_ = create_allignment_graph(s1,s2,table,5)
parse_while_allign(g,s1, s2)

[[0, -5, -10, -15, -20, -25, -30], [-5, -2, -6, -11, -16, -21, -26], [-10, -3, -5, -7, -12, -12, -17], [-15, -8, 2, -3, -7, -12, -14], [-20, -13, -3, 6, 1, -4, -9], [-25, -18, -8, 1, 7, 2, -3], [-30, -23, -13, -4, 2, 6, 1], [-35, -28, -18, -9, 2, 1, 4], [-40, -33, -23, -14, -3, 1, -1], [-45, -38, -28, -19, -8, 1, 0], [-50, -43, -33, -24, -13, -4, 8]]
PLEASANTLY
-MEA--N-LY


In [170]:
def read_seq_file(path):
    with open(path) as f:
        seq1 = f.readline().strip()
        seq2 = f.readline().strip()
    return seq1, seq2

In [171]:
def GlobalAllignment(seq1, seq2, penalty, table):
    all_graph, match_g = create_allignment_graph(seq1,seq2,table,penalty)
    print(match_g)
    parse_while_allign(all_graph,seq1, seq2)

In [187]:
path="/Users/marcisin/Downloads/dataset_247_3 (4).txt"
s1, s2 =read_seq_file(path)
# s1="PLEASANTLY"
# s2="MEANLY"
penalty = 5
GlobalAllignment(s1, s2, penalty, table)

1783
QMRAAWMHDKSQDAQKIFKHVYNP---IWGMPSFMGHPGSNWFNGEPFAHYMRFHQLADYGAAARIRLAGWNVEKVEVYPLQFHYWVDPI-V--PIHTVNMGSKGTCTTQVKNTCRFQMNIFFPSYYTIAHPATHAATLVHEFRCEKCQCPDRIYFPMKW---DKQYCRN-C---Y-V--A-MEG---KD--LGCTELNCIAWEAPFCCGLNSHRWILANDCKCWFERQKDDVVDDHHFHYENNLFFTGISSPEKAFWTQRNAEINTKWMIMR-----H--H-----VDIYQYAQKARMTS-KFNFGYIIFVNHSQMIIWPSR-RIMTDFYKQNISIATCDCSFSAAALTPRIDTTSVWQHPLCVDQRVMDSTLNRRGMASKCV---DPGDVNCRMGNF-----C--MEYMMNYEGHLN-T-----QMADY--T-Y--FYNWKAMMFPQGLIDTFVYMNTTRFPKEMLTKFWEPHFHVFLAFCWWGHY-GCHIPL-NV-CNKVGDVK-NFSHI-HPTS-FFALQHTM--WPN-QLSHAMYCTDEEHFVM-R---YFRWPWDTK-SQDG--SNTGPHQGF-W---K--AYMLKKE-Q-ICTLFETDNPGFPPAREGFKPANHKVKRVNH-EPWHYRTMMKYKYMWAVDTEPVRCSKMMSDKPKLTCTTQVEYSDITSKKDQEAQFFDMDPEKASRWFADYSHDCRYAVCDCKQSLEEYWVWLIPYWLDAIAEMVCFMQSSIRIWMAEDMVYFRNTF-S-----TIDKTMRHWWHLRKAHKGIQAMQQHAQQNGL--A--MNFTDCWFSN----GCFGTY-QDCMEIETGKSLTDFLLFKLMKLCGVILGIPAWIKNAILWRQPKVEQLCACENNWTAVHMSMDNYSQVSEFE
QMRAAWNQDWHQRLIKIFKHVYNQMLDIWDIMSFMLHPGWNWYM-RWLDPQQR-HILADYGAAWRIRLAGWRVEKVEGY--S-HYWSDPIPVIEPIHTVNMGS

In [184]:
GlobalAllignment('NPHLIYEWNRRFAGHTRSNFHG','YPRNQWVEPQASRVQSAH',5,table)

[[0, -5, -10, -15, -20, -25, -30, -35, -40, -45, -50, -55, -60, -65, -70, -75, -80, -85, -90], [-5, -2, -7, -10, -9, -14, -19, -24, -29, -34, -39, -44, -49, -54, -59, -64, -69, -74, -79], [-10, -7, 5, 0, -5, -10, -15, -20, -25, -22, -27, -32, -37, -42, -47, -52, -57, -62, -67], [-15, -8, 0, 5, 1, -4, -9, -14, -19, -24, -22, -27, -32, -37, -42, -47, -52, -57, -54], [-20, -13, -5, 0, 2, -1, -6, -8, -13, -18, -23, -23, -28, -33, -36, -41, -46, -51, -56], [-25, -18, -10, -5, -3, -1, -4, -3, -8, -13, -18, -23, -25, -30, -30, -35, -40, -45, -50], [-30, -18, -15, -10, -7, -4, 1, -4, -5, -10, -14, -19, -24, -27, -31, -31, -36, -41, -43], [-35, -23, -19, -15, -10, -5, -4, -1, 1, -4, -8, -13, -18, -23, -28, -29, -31, -36, -41], [-40, -28, -24, -20, -15, -10, 6, 1, -4, -3, -6, -11, -16, -21, -26, -30, -32, -34, -38], [-45, -33, -29, -24, -14, -15, 1, 3, 1, -4, -3, -8, -10, -15, -20, -25, -29, -34, -33], [-50, -38, -34, -24, -19, -13, -4, -2, 3, -1, -3, -4, -9, -5, -10, -15, -20, -25, -30], [-55, 